# NFL DFS Optimizer
#### Developed by Dan McDonough
#### September 8, 2021

The purpose of this notebook is to help construct an optimal DFS lineup for the week.

I don't have a whole lot of NFL domain knowledge, so I didn't want to spend time trying to build a player projection model. Instead, I wanted to try to build competitive advantage in my league by being able to maximize the number of projected points drafted, with a given set of projections taken as ground truth.

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np
from pulp import *

### 1. Scrape Projections

First, I need to scrape a source of fantasy projections. For my purposes, I am going to treat these projections as truth, and determine how to optimally select a lineup that maximizes the number of projected fantasy points.

#### QB Projections

In [2]:
def scrape_qb():
    url = 'https://www.fantasypros.com/nfl/projections/qb.php?week=1'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    attempts = []
    completions = []
    yards = []
    touchdowns = []
    interceptions = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    fumbles = []

    for i in range(66):
        name = len(str(soup.findAll('td')[4+11*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[4+11*i].findAll('a')[0]).split('>')[1][0:name-3])
        attempts.append(float(str(soup.findAll('td')[5+11*i]).split('>')[1].split('<')[0]))
        completions.append(float((str(soup.findAll('td')[6+11*i]).split('>')[1].split('<')[0])))
        yards.append(float(str(soup.findAll('td')[7+11*i]).split('>')[1].split('<')[0].split(',')[0]))
        touchdowns.append(float((str(soup.findAll('td')[8+11*i]).split('>')[1].split('<')[0])))
        interceptions.append(float((str(soup.findAll('td')[9+11*i]).split('>')[1].split('<')[0])))
        rush_attempts.append(float((str(soup.findAll('td')[10+11*i]).split('>')[1].split('<')[0])))
        rush_yards.append(float((str(soup.findAll('td')[11+11*i]).split('>')[1].split('<')[0])))
        rush_tds.append(float((str(soup.findAll('td')[12+11*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[13+11*i]).split('>')[1].split('<')[0])))
    
    qb_projections = pd.DataFrame({'player_stat':player_stat,'attempts':attempts,'completions':completions,'yards':yards,
                             'touchdowns':touchdowns,'interceptions':interceptions,'rush_attempts':rush_attempts,
                             'rush_yards':rush_yards,'rush_tds':rush_tds,'fumbles':fumbles})

    return qb_projections

In [3]:
qb_projections = scrape_qb()
qb_projections[0:10]

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles
0,Patrick Mahomes II,38.7,25.3,317.7,2.5,0.6,3.6,18.9,0.1,0.1
1,Josh Allen,36.0,23.0,268.1,2.1,0.8,7.1,34.7,0.3,0.2
2,Lamar Jackson,28.0,17.8,212.7,1.5,0.8,10.4,66.7,0.5,0.2
3,Kyler Murray,36.1,24.0,263.5,1.6,0.9,7.6,43.6,0.5,0.2
4,Tom Brady,38.3,24.6,296.3,2.5,0.7,1.5,3.0,0.1,0.1
5,Russell Wilson,34.7,23.1,271.9,2.0,0.6,4.8,27.8,0.1,0.2
6,Aaron Rodgers,35.9,23.7,279.6,2.1,0.5,2.7,13.1,0.1,0.2
7,Ryan Tannehill,31.9,21.0,253.2,2.0,0.5,3.6,18.0,0.2,0.1
8,Dak Prescott,39.5,26.0,294.3,1.7,0.9,3.7,16.9,0.2,0.2
9,Jalen Hurts,35.4,21.7,252.1,1.4,0.9,7.4,41.6,0.3,0.2


Next, I will add scoring for FanDuel

In [4]:
scoring_dict = {}
scoring_dict['passing_yards_score'] = 0.04
scoring_dict['passing_TD_score'] = 4
scoring_dict['passing_interception_score'] = -1
scoring_dict['rushing_yards_score'] = 0.1
scoring_dict['rushing_TD_score'] = 6
scoring_dict['offense_fumble_score'] = -2
scoring_dict['receptions_score'] = 0.5
scoring_dict['receiving_yards_score'] = 0.1
scoring_dict['receiving_TD_score'] = 6
scoring_dict['sack_score'] = 1
scoring_dict['defense_interception_score'] = 2
scoring_dict['defense_fumble_score'] = 2
scoring_dict['defense_TD_score'] = 6
scoring_dict['safety_score'] = 2
scoring_dict['points_against_score'] = -2/7
scoring_dict['fg_made_score'] = 3
scoring_dict['fg_miss_score'] = -1
scoring_dict['xp_made_score'] = 1
scoring_dict['xp_miss_score'] = -1

In [5]:
def qb_scoring(x, scoring_dict):
    points = x['yards'] * scoring_dict['passing_yards_score'] + x['touchdowns'] * scoring_dict['passing_TD_score'] +\
        x['interceptions'] * scoring_dict['passing_interception_score'] +\
        x['rush_yards'] * scoring_dict['rushing_yards_score'] + x['rush_tds'] * scoring_dict['rushing_TD_score'] +\
        x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
qb_projections['points'] = qb_projections.apply(qb_scoring, args =(scoring_dict, ), axis=1)

In [6]:
qb_projections.head(1)

,player_stat,attempts,completions,yards,touchdowns,interceptions,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Patrick Mahomes II,38.7,25.3,317.7,2.5,0.6,3.6,18.9,0.1,0.1,24.398


#### RB

In [7]:
def scrape_rb():
    url = 'https://www.fantasypros.com/nfl/projections/rb.php?week=1'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    receptions = []
    receive_yards = []
    receive_tds = []
    fumbles = []
    for i in range(161):
        name = len(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1][0:name-3])
        rush_attempts.append(float((str(soup.findAll('td')[5+9*i]).split('>')[1].split('<')[0])))
        if len(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]) < 6:
            rush_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0])))
        else:    
            rush_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]).split(',')[0]+(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]).split(',')[1]))
        rush_tds.append(float((str(soup.findAll('td')[7+9*i]).split('>')[1].split('<')[0])))
        receptions.append(float((str(soup.findAll('td')[8+9*i]).split('>')[1].split('<')[0])))
        receive_yards.append(float((str(soup.findAll('td')[9+9*i]).split('>')[1].split('<')[0])))
        receive_tds.append(float((str(soup.findAll('td')[10+9*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[11+9*i]).split('>')[1].split('<')[0])))

    rb_projections = pd.DataFrame({'player_stat':player_stat,'rush_attempts':rush_attempts,'rush_yards':rush_yards,
                                   'rush_tds':rush_tds,'receptions':receptions,'receive_yards':receive_yards,
                                   'receive_tds':receive_tds,'fumbles':fumbles})
    
    return rb_projections

In [8]:
rb_projections = scrape_rb()
rb_projections[0:10]

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles
0,Christian McCaffrey,19.0,80.3,0.8,5.4,44.9,0.3,0.1
1,Dalvin Cook,19.9,97.3,0.8,3.7,32.0,0.1,0.2
2,Derrick Henry,21.9,107.1,0.8,1.5,11.8,0.1,0.1
3,Alvin Kamara,13.2,65.0,0.6,5.4,47.9,0.2,0.1
4,Aaron Jones,15.1,71.8,0.6,3.4,25.0,0.1,0.1
5,Saquon Barkley,15.1,69.2,0.5,3.5,28.9,0.1,0.1
6,Joe Mixon,16.9,71.8,0.6,3.1,24.6,0.1,0.1
7,Nick Chubb,16.4,83.2,0.6,1.9,14.4,0.1,0.1
8,Najee Harris,16.6,72.9,0.6,3.2,24.9,0.1,0.1
9,Austin Ekeler,10.8,49.4,0.2,5.3,43.5,0.4,0.1


In [9]:
def rb_scoring(x, scoring_dict):
    points = x['rush_yards'] * scoring_dict['rushing_yards_score'] + x['rush_tds'] * scoring_dict['rushing_TD_score'] +\
    x['receptions'] * scoring_dict['receptions_score'] + x['receive_yards'] * scoring_dict['receiving_yards_score'] +\
    x['receive_tds'] * scoring_dict['receiving_TD_score'] + x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
rb_projections['points'] = rb_projections.apply(rb_scoring, args= (scoring_dict,), axis=1)

In [10]:
rb_projections.head(1)

,player_stat,rush_attempts,rush_yards,rush_tds,receptions,receive_yards,receive_tds,fumbles,points
0,Christian McCaffrey,19.0,80.3,0.8,5.4,44.9,0.3,0.1,21.62


#### WR

In [11]:
def scrape_wr():
    url = 'https://www.fantasypros.com/nfl/projections/wr.php?week=1'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    receptions = []
    receive_yards = []
    receive_tds = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    fumbles = []
    for i in range(247):
        name = len(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[4+9*i].findAll('a')[0]).split('>')[1][0:name-3])
        receptions.append(float((str(soup.findAll('td')[5+9*i]).split('>')[1].split('<')[0])))
        if len(str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0]) < 6:
            receive_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0])))
        else:
            receive_yards.append(float((str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[6+9*i]).split('>')[1].split('<')[0].split(',')[1])))
        receive_tds.append(float((str(soup.findAll('td')[7+9*i]).split('>')[1].split('<')[0])))
        rush_attempts.append(float((str(soup.findAll('td')[8+9*i]).split('>')[1].split('<')[0])))
        rush_yards.append(float((str(soup.findAll('td')[9+9*i]).split('>')[1].split('<')[0])))
        rush_tds.append(float((str(soup.findAll('td')[10+9*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[11+9*i]).split('>')[1].split('<')[0])))

    wr_projections = pd.DataFrame({'player_stat':player_stat,'receptions':receptions,'receive_yards':receive_yards,
                                   'receive_tds':receive_tds,'rush_attempts':rush_attempts,'rush_yards':rush_yards,
                                   'rush_tds':rush_tds,'fumbles':fumbles})
    
    return wr_projections

In [12]:
wr_projections = scrape_wr()
wr_projections[0:10]

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles
0,Tyreek Hill,6.5,93.1,0.8,0.8,4.7,0.0,0.1
1,Davante Adams,7.4,93.0,0.8,0.0,0.0,0.0,0.1
2,Calvin Ridley,6.2,92.4,0.7,0.3,1.8,0.0,0.1
3,Stefon Diggs,6.8,89.0,0.7,0.2,1.1,0.0,0.1
4,A.J. Brown,5.2,80.1,0.7,0.1,1.0,0.0,0.0
5,DeAndre Hopkins,7.3,88.6,0.6,0.0,0.3,0.0,0.1
6,Justin Jefferson,5.9,89.4,0.5,0.1,0.1,0.0,0.1
7,D.K. Metcalf,5.1,81.1,0.7,0.0,0.1,0.0,0.1
8,Adam Thielen,5.4,70.4,0.6,0.2,0.7,0.0,0.0
9,Tyler Lockett,6.0,72.8,0.6,0.1,0.4,0.0,0.1


In [13]:
def wr_scoring(x, scoring_dict):
    points = x['rush_yards'] * scoring_dict['rushing_yards_score'] + x['rush_tds'] * scoring_dict['rushing_TD_score'] +\
    x['receptions'] * scoring_dict['receptions_score'] + x['receive_yards'] * scoring_dict['receiving_yards_score'] +\
    x['receive_tds'] * scoring_dict['receiving_TD_score'] + x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
wr_projections['points'] = wr_projections.apply(wr_scoring, args= (scoring_dict,), axis=1)

In [14]:
wr_projections.head(1)

,player_stat,receptions,receive_yards,receive_tds,rush_attempts,rush_yards,rush_tds,fumbles,points
0,Tyreek Hill,6.5,93.1,0.8,0.8,4.7,0.0,0.1,17.63


#### TE

In [15]:
def scrape_te():
    url = 'https://www.fantasypros.com/nfl/projections/te.php?week=1'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    player_stat = []
    receptions = []
    receive_yards = []
    receive_tds = []
    rush_attempts = []
    rush_yards = []
    rush_tds = []
    fumbles = []
    for i in range(118):
        name = len(str(soup.findAll('td')[3+6*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[3+6*i].findAll('a')[0]).split('>')[1][0:name-3])
        receptions.append(float((str(soup.findAll('td')[4+6*i]).split('>')[1].split('<')[0])))
        if len(str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0]) < 6:
            receive_yards.append(float((str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0])))
        else:
            receive_yards.append(float(str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0].split(',')[0]+str(soup.findAll('td')[5+6*i]).split('>')[1].split('<')[0].split(',')[1]))
        receive_tds.append(float((str(soup.findAll('td')[6+6*i]).split('>')[1].split('<')[0])))
        fumbles.append(float((str(soup.findAll('td')[7+6*i]).split('>')[1].split('<')[0])))

    te_projections = pd.DataFrame({'player_stat':player_stat,'receptions':receptions,'receive_yards':receive_yards,
                                   'receive_tds':receive_tds,'fumbles':fumbles})
    
    return te_projections

In [16]:
te_projections = scrape_te()
te_projections[0:10]

,player_stat,receptions,receive_yards,receive_tds,fumbles
0,Travis Kelce,7.0,90.7,0.7,0.1
1,Darren Waller,6.6,78.5,0.5,0.1
2,George Kittle,5.6,74.0,0.5,0.0
3,Mark Andrews,4.8,57.1,0.6,0.0
4,T.J. Hockenson,4.5,50.7,0.4,0.0
5,Kyle Pitts,3.9,47.7,0.4,0.0
6,Logan Thomas,4.5,45.1,0.4,0.0
7,Mike Gesicki,4.0,45.7,0.4,0.0
8,Dallas Goedert,4.0,44.6,0.4,0.0
9,Robert Tonyan,3.4,37.7,0.4,0.0


In [17]:
def te_scoring(x, scoring_dict):
    points = x['receptions'] * scoring_dict['receptions_score'] +\
    x['receive_yards'] * scoring_dict['receiving_yards_score'] + x['receive_tds'] * scoring_dict['receiving_TD_score'] +\
    x['fumbles'] * scoring_dict['offense_fumble_score']
    
    return points
    
te_projections['points'] = te_projections.apply(te_scoring, args= (scoring_dict,), axis=1)

In [18]:
te_projections.head(1)

,player_stat,receptions,receive_yards,receive_tds,fumbles,points
0,Travis Kelce,7.0,90.7,0.7,0.1,16.57


#### D/ST

In [19]:
def scrape_dst():
    url = 'https://www.fantasypros.com/nfl/projections/dst.php?week=1'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    player_stat = []
    sack = []
    interception = []
    fumble_recovered = []
    touchdowns = []
    safety = []
    points_against = []
    yards_against = []

    for i in range(32):
        name = len(str(soup.findAll('td')[0+10*i].findAll('a')[0]).split('>')[1])
        player_stat.append(str(soup.findAll('td')[0+10*i].findAll('a')[0]).split('>')[1][0:name-3])
        sack.append(float((str(soup.findAll('td')[1+10*i]).split('>')[1].split('<')[0])))
        interception.append(float((str(soup.findAll('td')[2+10*i]).split('>')[1].split('<')[0])))
        fumble_recovered.append(float((str(soup.findAll('td')[3+10*i]).split('>')[1].split('<')[0])))
        touchdowns.append(float((str(soup.findAll('td')[5+10*i]).split('>')[1].split('<')[0])))
        safety.append(float((str(soup.findAll('td')[6+10*i]).split('>')[1].split('<')[0])))
        points_against.append(float((str(soup.findAll('td')[7+10*i]).split('>')[1].split('<')[0])))
        yards_against.append((str(soup.findAll('td')[8+10*i]).split('>')[1].split('<')[0]))

    dst_projections = pd.DataFrame({'player_stat':player_stat,'sack':sack,'interception':interception,
                                   'fumble_recovered':fumble_recovered,'touchdowns':touchdowns,
                                   'safety':safety,'points_against':points_against,'yards_against':yards_against})
    
    return dst_projections

In [20]:
dst_projections = scrape_dst()
dst_projections[0:10]

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against
0,Los Angeles Rams,3.1,1.0,0.6,0.2,0.1,18.9,282.4
1,New England Patriots,2.7,1.0,0.5,0.1,0.0,20.0,326.0
2,Denver Broncos,3.3,1.0,0.7,0.2,0.0,20.5,295.3
3,San Francisco 49ers,2.6,0.9,0.6,0.1,0.0,18.9,292.5
4,Carolina Panthers,2.5,0.9,0.6,0.1,0.1,19.4,308.0
5,Green Bay Packers,2.6,1.2,0.6,0.1,0.0,22.8,334.0
6,Indianapolis Colts,3.0,0.9,0.6,0.1,0.1,23.8,351.9
7,Philadelphia Eagles,3.2,0.8,0.6,0.1,0.0,24.8,363.4
8,Seattle Seahawks,2.9,0.9,0.6,0.1,0.1,22.8,317.8
9,Minnesota Vikings,2.9,0.9,0.7,0.1,0.0,22.6,356.7


In [21]:
def dst_scoring(x, scoring_dict):
    points = x['sack'] * scoring_dict['sack_score'] + x['interception'] * scoring_dict['defense_interception_score'] +\
    x['fumble_recovered'] * scoring_dict['defense_fumble_score'] + x['touchdowns'] * scoring_dict['defense_TD_score'] +\
    x['safety'] * scoring_dict['safety_score'] + x['points_against'] * scoring_dict['points_against_score'] + 7
    
    return points
    
dst_projections['points'] = dst_projections.apply(dst_scoring, args= (scoring_dict,), axis=1)

In [22]:
dst_projections.head(1)

,player_stat,sack,interception,fumble_recovered,touchdowns,safety,points_against,yards_against,points
0,Los Angeles Rams,3.1,1.0,0.6,0.2,0.1,18.9,282.4,9.3


#### Combine projections

In [23]:
all_projections = pd.concat([qb_projections[['player_stat','points']],rb_projections[['player_stat','points']],wr_projections[['player_stat','points']],
          te_projections[['player_stat','points']],dst_projections[['player_stat','points']]])

all_projections.sort_values(by='points',ascending=False,inplace=True)
all_projections.head()

,player_stat,points
0,Patrick Mahomes II,24.398
1,Josh Allen,23.194
3,Kyler Murray,23.000
2,Lamar Jackson,22.978
4,Tom Brady,21.852


### 2. Load salaries

Now I will load the player salaries from FanDuel.

In [24]:
df_raw = pd.read_csv('players.csv')
df = df_raw[df_raw['Injury Indicator'].apply(lambda x: False if x in ['IR','O'] else True)].copy()
df['FPPG'] = df['FPPG'].fillna(0)
df.head(2)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Roster Position
0,63499-55050,RB,Christian,Christian McCaffrey,McCaffrey,16.380000,5.0,10400,NYJ@CAR,CAR,NYJ,NaN,NaN,NaN,RB/FLEX
1,63499-54140,RB,Dalvin,Dalvin Cook,Cook,22.557142,14.0,9400,MIN@CIN,MIN,CIN,NaN,NaN,NaN,RB/FLEX


First, I need to standardize names across data sets

In [25]:
def check_players(x):
    print("In projections, but not in salaries: ")
    for index, player in enumerate(x['player_stat']):
        if player not in df_raw['Nickname'].unique():        
            print(index,' ',player)
    test_df = df_raw.merge(x,how='left',left_on='Nickname',right_on='player_stat')
    missing_df = test_df[(test_df['player_stat'].isna())]
    if len(missing_df) > 0:
        print('\nIn salaries, but not in projections:')
        return missing_df

In [26]:
check_players(all_projections)

In projections, but not in salaries: 
0   Patrick Mahomes II
50   D.K. Metcalf
75   D.J. Moore
96   Darrell Henderson
104   D.J. Chark Jr.
229   D'Wayne Eskridge
235   Van Jefferson
242   Josh Palmer
258   Chris Herndon IV
286   Le'Veon Bell
352   Jakeem Grant Sr.
394   Ray-Ray McCloud
427   Duke Johnson Jr.
428   Danny Amendola
429   J.J. Arcega-Whiteside
432   Dwayne Haskins
435   Phillip Walker
450   James Proche II
451   K.J. Hill Jr.
460   Stanley Morgan Jr.
463   Elijah Mitchell
496   Victor Bolden Jr.
501   Mike White
519   John Ross
520   Godwin Igwebuike
537   Alex Armah Jr.
546   Jason Cabinda
558   Elijhaa Penny
578   Cyril Grayson Jr.
590   Jake Lampman
605   Derrick Willies
608   Adrian Killins Jr.

In salaries, but not in projections:


,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Roster Position,player_stat,points
3,63499-57439,QB,Patrick,Patrick Mahomes,Mahomes,24.422223,18.0,8800,CLE@KC,KC,CLE,NaN,NaN,NaN,QB,NaN,NaN
23,63499-73048,WR,DK,DK Metcalf,Metcalf,14.935294,17.0,7700,SEA@IND,SEA,IND,NaN,NaN,NaN,WR/FLEX,NaN,NaN
45,63499-14185,QB,Cam,Cam Newton,Newton,17.998667,15.0,7000,MIA@NE,NE,MIA,NaN,NaN,NaN,QB,NaN,NaN
61,63499-63043,WR,DJ,DJ Moore,Moore,11.900000,15.0,6700,NYJ@CAR,CAR,NYJ,NaN,NaN,NaN,WR/FLEX,NaN,NaN
76,63499-40167,RB,Gus,Gus Edwards,Edwards,7.316666,18.0,6500,BAL@LV,BAL,LV,NaN,NaN,NaN,RB/FLEX,NaN,NaN
81,63499-28573,WR,Michael,Michael Thomas,Thomas,8.855555,9.0,6500,GB@NO,NO,GB,NaN,Ankle,NaN,WR/FLEX,NaN,NaN
82,63499-70157,RB,Darrell,Darrell Henderson Jr.,Henderson Jr.,8.153334,15.0,6400,CHI@LAR,LAR,CHI,Q,Thumb,NaN,RB/FLEX,NaN,NaN
90,63499-92560,QB,Sam,Sam Ehlinger,Ehlinger,NaN,NaN,6300,SEA@IND,IND,SEA,IR,Knee,NaN,QB,NaN,NaN
95,63499-32158,QB,Garrett,Garrett Gilbert,Gilbert,7.760000,2.0,6300,MIA@NE,NE,MIA,NaN,NaN,NaN,QB,NaN,NaN
99,63499-31047,QB,AJ,AJ McCarron,McCarron,0.400000,2.0,6200,PHI@ATL,ATL,PHI,IR,Knee - acl,NaN,QB,NaN,NaN


In [27]:
all_projections['player_stat'].replace('Patrick Mahomes II','Patrick Mahomes',inplace=True)
all_projections['player_stat'].replace('D.K. Metcalf','DK Metcalf',inplace=True)
all_projections['player_stat'].replace('D.J. Moore','DJ Moore',inplace=True)
all_projections['player_stat'].replace('Darrell Henderson','Darrell Henderson Jr.',inplace=True)
all_projections['player_stat'].replace('D.J. Chark Jr.','DJ Chark Jr.',inplace=True)
all_projections['player_stat'].replace("D'Wayne Eskridge",'Dee Eskridge',inplace=True)
all_projections['player_stat'].replace('Van Jefferson','Van Jefferson Jr.',inplace=True)
all_projections['player_stat'].replace('Josh Palmer','Joshua Palmer',inplace=True)
all_projections['player_stat'].replace('Chris Herndon IV','Chris Herndon',inplace=True)
all_projections['player_stat'].replace('Jakeem Grant Sr.','Jakeem Grant',inplace=True)
all_projections['player_stat'].replace('Ray-Ray McCloud','Ray-Ray McCloud III',inplace=True)
all_projections['player_stat'].replace('Phillip Walker','P.J. Walker',inplace=True)
all_projections['player_stat'].replace('J.J. Arcega-Whiteside','JJ Arcega-Whiteside',inplace=True)
all_projections['player_stat'].replace('James Proche II','James Proche',inplace=True)
all_projections['player_stat'].replace('Stanley Morgan Jr.','Stanley Morgan',inplace=True)
all_projections['player_stat'].replace('K.J. Hill Jr.','K.J. Hill',inplace=True)
all_projections['player_stat'].replace('Victor Bolden Jr.','Victor Bolden',inplace=True)
all_projections['player_stat'].replace('Alex Armah Jr.','Alex Armah',inplace=True)
all_projections['player_stat'].replace('John Ross','John Ross III',inplace=True)
all_projections['player_stat'].replace('Elijhaa Penny','Eli Penny',inplace=True)
all_projections['player_stat'].replace('Elijah Mitchell','Eli Mitchell',inplace=True)
all_projections['player_stat'].replace('Adrian Killins Jr.','Adrian Killins',inplace=True)

In [28]:
check_players(all_projections)

In projections, but not in salaries: 
286   Le'Veon Bell
427   Duke Johnson Jr.
428   Danny Amendola
432   Dwayne Haskins
501   Mike White
520   Godwin Igwebuike
546   Jason Cabinda
578   Cyril Grayson Jr.
590   Jake Lampman
605   Derrick Willies

In salaries, but not in projections:


,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Roster Position,player_stat,points
45,63499-14185,QB,Cam,Cam Newton,Newton,17.998667,15.0,7000,MIA@NE,NE,MIA,NaN,NaN,NaN,QB,NaN,NaN
76,63499-40167,RB,Gus,Gus Edwards,Edwards,7.316666,18.0,6500,BAL@LV,BAL,LV,NaN,NaN,NaN,RB/FLEX,NaN,NaN
81,63499-28573,WR,Michael,Michael Thomas,Thomas,8.855555,9.0,6500,GB@NO,NO,GB,NaN,Ankle,NaN,WR/FLEX,NaN,NaN
90,63499-92560,QB,Sam,Sam Ehlinger,Ehlinger,NaN,NaN,6300,SEA@IND,IND,SEA,IR,Knee,NaN,QB,NaN,NaN
95,63499-32158,QB,Garrett,Garrett Gilbert,Gilbert,7.760000,2.0,6300,MIA@NE,NE,MIA,NaN,NaN,NaN,QB,NaN,NaN
99,63499-31047,QB,AJ,AJ McCarron,McCarron,0.400000,2.0,6200,PHI@ATL,ATL,PHI,IR,Knee - acl,NaN,QB,NaN,NaN
110,63499-61276,QB,James,James Morgan,Morgan,0.000000,1.0,6200,NYJ@CAR,CAR,NYJ,NaN,NaN,NaN,QB,NaN,NaN
119,63499-52606,QB,David,David Blough,Blough,1.380000,2.0,6100,SF@DET,DET,SF,NaN,NaN,NaN,QB,NaN,NaN
121,63499-38767,QB,Chris,Chris Streveler,Streveler,2.175000,4.0,6100,ARI@TEN,ARI,TEN,NaN,NaN,NaN,QB,NaN,NaN
123,63499-71710,QB,Nate,Nate Stanley,Stanley,NaN,NaN,6000,MIN@CIN,MIN,CIN,IR,Undisclosed,NaN,QB,NaN,NaN


In [29]:
combined_df = df.merge(all_projections,how='left',left_on='Nickname',right_on='player_stat')
combined_df['points'] = combined_df['points'].fillna(0)

### 3. Build optimizer

First, I will create the necessary data structures

In [30]:
prob = LpProblem("combined_dfS",LpMaximize)

players = list(combined_df['Id'])
salaries = dict(zip(players,combined_df['Salary']))
points = dict(zip(players,combined_df['points']))
qbs = dict(zip(players,(combined_df['Position']=='QB')*1))
rbs = dict(zip(players,(combined_df['Position']=='RB')*1))
wrs = dict(zip(players,(combined_df['Position']=='WR')*1))
tes = dict(zip(players,(combined_df['Position']=='TE')*1))
defs = dict(zip(players,(combined_df['Position']=='D')*1))
player_count = dict(zip(players,[1]*len(combined_df)))

player_vars = LpVariable.dicts("Players",players,lowBound=0,upBound=1,cat='Integer')

Next, I will define the objective and constraints

In [31]:
prob += lpSum([points[i]*player_vars[i] for i in players])

prob += lpSum([salaries[f] * player_vars[f] for f in players]) <= 60000

prob += lpSum([qbs[f] * player_vars[f] for f in players]) == 1 
prob += lpSum([rbs[f] * player_vars[f] for f in players]) >= 2
prob += lpSum([rbs[f] * player_vars[f] for f in players]) <= 3
prob += lpSum([wrs[f] * player_vars[f] for f in players]) >= 3
prob += lpSum([wrs[f] * player_vars[f] for f in players]) <= 4
prob += lpSum([tes[f] * player_vars[f] for f in players]) >= 1
prob += lpSum([tes[f] * player_vars[f] for f in players]) <= 2
prob += lpSum([defs[f] * player_vars[f] for f in players]) == 1
prob += lpSum([player_count[f] * player_vars[f] for f in players]) == 9

Solve problem

In [32]:
prob.solve()

1

In [33]:
players_out = []

for v in prob.variables():
    if v.varValue>0:
        players_out.append('-'.join(str(v).split('_')[1:]))
        
combined_df['keep'] = combined_df['Id'].apply(lambda x: 1 if x in players_out else 0)

In [34]:
combined_df[combined_df['keep']==1]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Roster Position,player_stat,points,keep
3,63499-57439,QB,Patrick,Patrick Mahomes,Mahomes,24.422223,18.0,8800,CLE@KC,KC,CLE,NaN,NaN,NaN,QB,Patrick Mahomes,24.398000,1
8,63499-53681,WR,Tyreek,Tyreek Hill,Hill,18.572222,18.0,8500,CLE@KC,KC,CLE,NaN,NaN,NaN,WR/FLEX,Tyreek Hill,17.630000,1
42,63499-25011,TE,Darren,Darren Waller,Waller,14.068750,16.0,7000,BAL@LV,LV,BAL,NaN,NaN,NaN,TE/FLEX,Darren Waller,13.950000,1
47,63499-80001,RB,Austin,Austin Ekeler,Ekeler,12.572728,11.0,6900,LAC@WAS,LAC,WAS,NaN,NaN,NaN,RB/FLEX,Austin Ekeler,15.340000,1
72,63499-90575,RB,Najee,Najee Harris,Harris,0.000000,NaN,6500,PIT@BUF,PIT,BUF,NaN,NaN,NaN,RB/FLEX,Najee Harris,15.380000,1
97,63499-79970,WR,Cooper,Cooper Kupp,Kupp,10.781250,16.0,6200,CHI@LAR,LAR,CHI,NaN,NaN,NaN,WR/FLEX,Cooper Kupp,13.210000,1
182,63499-89981,WR,Tee,Tee Higgins,Higgins,10.740000,15.0,6000,MIN@CIN,CIN,MIN,NaN,NaN,NaN,WR/FLEX,Tee Higgins,13.040000,1
189,63499-130288,RB,James,James Robinson,Robinson,16.135714,14.0,5900,JAC@HOU,JAC,HOU,NaN,NaN,NaN,RB/FLEX,James Robinson,13.770000,1
804,63499-12531,D,Denver,Denver Broncos,Broncos,5.062500,16.0,4100,DEN@NYG,DEN,NYG,NaN,NaN,NaN,DEF,Denver Broncos,9.042857,1


In [35]:
combined_df[combined_df['keep']==1]['Salary'].sum()

59900

In [36]:
combined_df[combined_df['keep']==1]['FPPG'].sum()

112.35538629